In [1]:
import open3d.ml as _ml3d
import open3d.ml.torch as ml3d
import pandas as pd
import numpy as np
import os

In [2]:
# Load configuration and model
cfg_file = "/Users/muhammadnumanmuttaqi/Documents/MScITBE/Semester2/E_SoftwareLab/SL/Open3D-ML-main/ml3d/configs/randlanet_s3dis.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

In [3]:
# Load dataset and pipeline
model = ml3d.models.RandLANet(**cfg.model)
cfg.dataset['dataset_path'] = "S3DIS"
dataset = ml3d.datasets.S3DIS(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.SemanticSegmentation(model, dataset=dataset, device='gpu', **cfg.pipeline)


In [4]:
# Download and load pretrained weights
ckpt_folder = "/Users/muhammadnumanmuttaqi/Documents/MScITBE/Semester2/SoftwareLab/SL/logs/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = "/Users/muhammadnumanmuttaqi/Documents/MScITBE/Semester2/E_SoftwareLab/SL/logs/randlanet_s3dis_202201071330utc.pth"

# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)


In [7]:
# Retrieve the test split to get information of the data (room names) by their index
test_split = dataset.get_split("test")

# Print attributes of the first datum
for i in range(len(test_split)):
    # Retrieve each sample
    data = test_split.get_attr(i)

    print("attributes: ", data)


attributes:  {'idx': 0, 'name': 'Area_5_storage_4', 'path': 'S3DIS/original_pkl/Area_5_storage_4.pkl', 'split': 'test'}
attributes:  {'idx': 1, 'name': 'Area_5_hallway_11', 'path': 'S3DIS/original_pkl/Area_5_hallway_11.pkl', 'split': 'test'}
attributes:  {'idx': 2, 'name': 'Area_5_hallway_10', 'path': 'S3DIS/original_pkl/Area_5_hallway_10.pkl', 'split': 'test'}
attributes:  {'idx': 3, 'name': 'Area_5_hallway_12', 'path': 'S3DIS/original_pkl/Area_5_hallway_12.pkl', 'split': 'test'}
attributes:  {'idx': 4, 'name': 'Area_5_hallway_13', 'path': 'S3DIS/original_pkl/Area_5_hallway_13.pkl', 'split': 'test'}
attributes:  {'idx': 5, 'name': 'Area_5_storage_2', 'path': 'S3DIS/original_pkl/Area_5_storage_2.pkl', 'split': 'test'}
attributes:  {'idx': 6, 'name': 'Area_5_storage_3', 'path': 'S3DIS/original_pkl/Area_5_storage_3.pkl', 'split': 'test'}
attributes:  {'idx': 7, 'name': 'Area_5_storage_1', 'path': 'S3DIS/original_pkl/Area_5_storage_1.pkl', 'split': 'test'}
attributes:  {'idx': 8, 'name': 

In [8]:
# Checking the accuracy for one sample data
# Load the test split and retrieve specific sample
test_split = dataset.get_split("test")
data = test_split.get_data(19) #office_10

# Run inference on a single example.
result = pipeline.run_inference(data)

# Retrieve predicted labels and ground truth labels
predicted_labels = result["predict_labels"]
ground_truth_labels = data["label"]

# Calculate the total number of points and the number of correctly predicted points
total_points = len(ground_truth_labels)
correct_predictions = (predicted_labels == ground_truth_labels).sum()

# Calculate accuracy
accuracy = correct_predictions / total_points

print(f"Accuracy for this sample: {accuracy * 100:.2f}%")

test 0/1: 100%|█████████▉| 52713/52805 [00:02<00:00, 24874.33it/s]

Accuracy for this sample: 92.86%


test 0/1: 100%|██████████| 52805/52805 [00:19<00:00, 24874.33it/s]

In [ ]:
# Step 1: Load the test split
test_split = dataset.get_split("test")

# Step 2: Loop through all samples in the test split
for i in range(len(test_split)):
    # Retrieve each sample
    data = test_split.get_data(i)
    
    # Run inference on the sample
    result = pipeline.run_inference(data)
    
    # Combine point cloud coordinates and predicted labels
    points = data["point"]  # 3D coordinates (x, y, z)
    labels = result["predict_labels"]  # Predicted labels
    
    # Prepare data for classification in a structured format
    classification_data = {
        "x": points[:, 0],
        "y": points[:, 1],
        "z": points[:, 2],
        "label": labels  # Predicted semantic segmentation labels
    }
    
    df = pd.DataFrame(classification_data)
    
    # Step 3: Class mapping
    class_mapping = {
        0: "ceiling",
        1: "floor",
        2: "wall",
        3: "beam",
        4: "column",
        5: "window",
        6: "door",
        7: "table",
        8: "chair",
        9: "sofa",
        10: "bookcase",
        11: "board",
        12: "clutter"
    }
    
    # Add object names based on the class_mapping dictionary
    df["object_name"] = df["label"].map(class_mapping)

    # Group by object_name to get only one instance per class as requested from classification team
    unique_object_names_df = df[["object_name"]].drop_duplicates().reset_index(drop=True)
    
    # Step 4: Save each room's unique object names to its own CSV file
    output_csv_path = f"room_{i}_unique_objects.csv"  # Unique filename for each room
    unique_object_names_df.to_csv(output_csv_path, index=False)
    print(f"Unique object names for room {i} saved to {output_csv_path}")

    # We changed the name from the room index to the name of the room manually

test 0/1: 100%|██████████| 48110/48110 [00:03<00:00, 10267.61it/s]

Unique object names for room 0 saved to room_0_unique_objects.csv


test 0/1: 100%|██████████| 48110/48110 [00:07<00:00, 6548.11it/s] 


Unique object names for room 1 saved to room_1_unique_objects.csv


test 0/1: 100%|██████████| 78356/78356 [00:06<00:00, 10892.20it/s]

...

Unique object names for room 65 saved to room_65_unique_objects.csv


test 0/1: 100%|██████████| 219688/219688 [01:18<00:00, 1070.33it/s]

Unique object names for room 66 saved to room_66_unique_objects.csv


test 0/1: 100%|██████████| 219688/219688 [01:26<00:00, 2539.76it/s]


Unique object names for room 67 saved to room_67_unique_objects.csv


In [ ]:
# Record the result of the performance
# Manual logging
import logging

# Setup logging
log_file_path = os.path.join(ckpt_folder, 'evaluation.log')
logging.basicConfig(filename=log_file_path, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Log inference result
logging.info("Inference result: %s", result)

# Evaluate performance on the test set
performance_metrics = pipeline.run_test()

# Output to console for verification
print("Performance Metrics:", performance_metrics)
print(f"Log is written in {log_file_path}")

test 66/68: 100%|██████████| 219688/219688 [01:30<00:00, 2429.92it/s]


Performance Metrics: None
Log is written in /Users/muhammadnumanmuttaqi/Documents/MScITBE/Semester2/SoftwareLab/SL/logs/evaluation.log


In [5]:
# Visualize the dataset with open3D-ML visualizer
# get the 'all' split that combines training, validation and test set
all_split = dataset.get_split('all')

# print the attributes of the first datum
print(all_split.get_attr(0))

# print the shape of the first point cloud
print(all_split.get_data(0)['point'].shape)

# show the first 100 frames using the visualizer
vis = ml3d.vis.Visualizer()
vis.visualize_dataset(dataset, 'all', indices=range(100))

{'idx': 0, 'name': 'Area_3_hallway_6', 'path': 'S3DIS/original_pkl/Area_3_hallway_6.pkl', 'split': 'all'}
(207314, 3)
FEngine (64 bits) created at 0x2b6a00000 (threading is enabled)
FEngine resolved backend: OpenGL


[error] GLFW error: Cocoa: Failed to find service port for display


: 